In [4]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\OrbitalNeuralMethods`


In [5]:
n = 2
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 10
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

rhf = RHF(system);
rhf = compute_ground_state!(rhf, max_iter = 100, tol = 1e-10, verbose=0);
slater = Slater(rhf);

In [98]:
nn = NeuralNetwork([Dense(32), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);

metro = Importance(equils = 0, samples = 1e6, step = 0.01);

In [103]:
@time result = blocking(wf, ham, metro, nthreads = 12);

  0.586429 seconds (2.23 k allocations: 44.730 MiB)


In [ ]:
0.463516 seconds (2.23 k allocations: 44.649 MiB)

In [37]:
optimizer = ADAM(wf, lr = 0.01, max_iter = 1, tol = 1e-6);
@time wf_final, wf_opt, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = false);

  0.971832 seconds (2.62 k allocations: 724.062 KiB)
